In [ ]:
import numpy
from isicarchive import jitfunc
from isicarchive.api import IsicApi
from isicarchive.font import Font
from isicarchive.sampler import Sampler, _sample_grid_2d
from matplotlib import pyplot
%matplotlib inline

f = Font(None)
s = Sampler()
k = s._kernels['lanczos3']

api = IsicApi()
image = api.image('ISIC_0000000')
image.load_image_data()
rimage = api.resample_image(image.data, (800,1200))
api.set_text_in_image(rimage, 'Test', fsize=0.04, min_alpha=1.0, color=[0,0,0], bcolor=[255,255,255])

pyplot.figure(figsize=(12,8))
pyplot.imshow(rimage)
pyplot.show()

In [ ]:
import timeit
%timeit rimage = api.resample_image(image.data, (400,600))

In [ ]:
skf = 3.0
sk = jitfunc.conv_kernel(skf * float(k[1])).astype(numpy.float64)
skl = sk.size
skr = (skl - 1) // (2 * k[1])
skr = 2 * k[1] * skr + 1
skd = (skl - skr) // 2
sk = sk[skd:skr+skd]
sk = sk / numpy.sum(sk)
ksk = numpy.convolve(k[0], sk)
while numpy.sum(ksk[0:k[1]]) < 0.01:
    ksk = ksk[k[1]:-k[1]]
print(k[0].size)
print(sk.size)
print(ksk.size)
pyplot.figure(figsize=(16,5))
pyplot.subplot(1, 3, 1)
pyplot.plot(k[0])
pyplot.subplot(1, 3, 2)
pyplot.plot(sk)
pyplot.subplot(1, 3, 3)
pyplot.plot(ksk)
pyplot.show()

In [ ]:
tdata = numpy.zeros(40000, dtype=numpy.uint8).reshape((200,200,))
tdata[:, 4:8] = 255
tdata[:, 20:22] = 255
tdata[:, 32:64:8] = 128
tdata[:, 33:65:8] = 255
tdata[:, 80:120:8] = 255
tdata[:, 81:121:8] = 128
tdata[:, 144:160] = 192
tdata[14:18,:] = 128
tdata[30:32,:] = 128
tdata[48:80:8,:] = 64
tdata[49:81:8,:] = 128
tdata[96:136:8,:] = 128
tdata[97:137:8,:] = 64
tdata[160:176,:] = 96
pyplot.figure(figsize=(8,8))
pyplot.imshow(tdata)
pyplot.show()

In [ ]:
(fi, fa) = f.set_text('!!TEST!!', 72.0)
fau = (255.0 * fa).astype(numpy.uint8)
pyplot.imshow(fau)
pyplot.show()

In [ ]:
tcopy = numpy.copy(tdata)
tcopy[60:60+fau.shape[0], 10:10+fau.shape[1]] = numpy.maximum(
    tcopy[60:60+fau.shape[0], 10:10+fau.shape[1]], fau)
pyplot.figure(figsize=(8,8))
pyplot.imshow(tcopy)
pyplot.show()

In [ ]:
fy = 1.7
fx = 1.7
sy = 3.5
sx = 3.5
ty = 199.5
tx = 199.5
tcopy.shape = (tcopy.shape[0], tcopy.shape[1])
rdata = _sample_grid_2d(tcopy, numpy.arange(fy,ty,sy), numpy.arange(fx,tx,sx), ksk, k[1])
rdata = numpy.maximum(numpy.minimum(rdata, 255.0), 0.0)
pyplot.figure(figsize=(8,8))
pyplot.imshow(rdata)
pyplot.show()

In [ ]:
from isicarchive.imfunc import image_resample
rdata2 = image_resample(tcopy, rdata.shape)
pyplot.figure(figsize=(8,8))
pyplot.imshow(rdata2)
pyplot.show()

In [ ]:
import timeit
tcopy.shape = (tcopy.shape[0], tcopy.shape[1])
%timeit rdata = _sample_grid_2d(tcopy, numpy.arange(fy,ty,sy), numpy.arange(fx,tx,sx), ksk, k[1])

In [ ]:
%timeit rdata2 = image_resample(tcopy, rdata.shape)